In [1]:
library(Seurat)
library(scater)

Attaching SeuratObject

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, r

In [21]:
source('../my_colors.R')

In [2]:
load('output2_step2.5.filter_junk/TotalTissue.combined.not.doubletFiltered.RData')

In [3]:
metaobs=read.csv('output2_step2.5.3.filter_junk_scanpy_pipeline/data_Epithelial/output_infercnv/cnv.summary.all.cells.combined.pubdata.csv',
                header=T, row.names=1)

In [4]:
ls()

[1] "metaobs"              "TotalTissue.combined"

In [7]:
rownames(metaobs)=metaobs$id

In [8]:
head(metaobs)

,id,cnv,cnv.cor,Sample,Defined_Cell_Type,type,malignant,cell_assignment
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
AAACCTGAGACGACGT_LUNG_T09,AAACCTGAGACGACGT_LUNG_T09,NA,NA,LUNG_T09,Myeloid cells,immu.stroma,NA,immu.stroma
AAACCTGAGACGACGT_LUNG_T34,AAACCTGAGACGACGT_LUNG_T34,0.02902300,-0.047078370,LUNG_T34,B cells,immu.stroma,non-malignant,immu.stroma
AAACCTGAGACGCTTT_LUNG_T18,AAACCTGAGACGCTTT_LUNG_T18,0.06901403,0.384063139,LUNG_T18,Epithelial,Epi,malignant,malignant
AAACCTGAGACTGGGT-1-6,AAACCTGAGACTGGGT-1-6,0.07265431,0.713340585,E3,Epithelial,Epi,malignant,malignant
AAACCTGAGAGGGCTT-1-6,AAACCTGAGAGGGCTT-1-6,0.01025899,-0.017148673,E3,Myeloid cells,immu.stroma,non-malignant,immu.stroma
AAACCTGAGATATGCA_LUNG_T25,AAACCTGAGATATGCA_LUNG_T25,0.05198921,0.004436815,LUNG_T25,T cells,immu.stroma,non-malignant,immu.stroma


In [9]:
all(rownames(metaobs) %in% colnames(TotalTissue.combined))

[1] TRUE

In [11]:
table(metaobs$Sample, metaobs$cell_assignment)

          
            Epi immu.stroma malignant
  E1        328        2982       670
  E2         19        2568      1945
  E3        148        1098      3088
  E4        120        3379       172
  LUNG_T06   12        2529        71
  LUNG_T08    8        2863        21
  LUNG_T09    0        2883         0
  LUNG_T18    0        1881       611
  LUNG_T19   21        3149       154
  LUNG_T20    1        2138       262
  LUNG_T25   15        3039       177
  LUNG_T28    4        1846       689
  LUNG_T30   21        1686       638
  LUNG_T31   25        2178        61
  LUNG_T34  292         472      1823
  Y1        174        2847        62
  Y2         84        2608       412
  Y3        696        1322       668
  Y4        165        2156       746

In [14]:
#select only malignant cells with cells numbers at least 50
#delete LUNG_T08 LUNG_T09
#
metaobs_sel=subset(metaobs, cell_assignment== 'malignant')
metaobs_sel=subset(metaobs_sel, !(Sample %in% c('LUNG_T08', 'LUNG_T09')))


In [15]:
table(metaobs_sel$Sample, metaobs_sel$cell_assignment)

          
           malignant
  E1             670
  E2            1945
  E3            3088
  E4             172
  LUNG_T06        71
  LUNG_T18       611
  LUNG_T19       154
  LUNG_T20       262
  LUNG_T25       177
  LUNG_T28       689
  LUNG_T30       638
  LUNG_T31        61
  LUNG_T34      1823
  Y1              62
  Y2             412
  Y3             668
  Y4             746

In [16]:
all(rownames(metaobs_sel) %in% colnames(TotalTissue.combined))

[1] TRUE

In [17]:
sce=subset(TotalTissue.combined, cells = rownames(metaobs_sel))

In [18]:
sce

An object of class Seurat 
20532 features across 12249 samples within 1 assay 
Active assay: RNA (20532 features, 2000 variable features)

In [19]:
head(sce@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,Index,Sample,percent.mt,percent.rb,Cell_type,Cell_type.refined,Cell_subtype,⋯,Phase,old.ident,CC.Difference,Cell_subtype.unimodel,Cell_type.unimodel,UMAP_Clusters.harmony_overclustered,scrubletScore_default,scrublet_predict_default,scrubletScore_n30,scrublet_predict_n30
,<chr>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<fct>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<chr>,<dbl>,<chr>
AAACCTGAGACGCTTT_LUNG_T18,pubsce,5243,867,AAACCTGAGACGCTTT_LUNG_T18,LUNG_T18,0.1883594,1.130156,Epithelial cells,Epithelial cells,tS2,⋯,G1,LUNG_T18,-0.004011074,tS2,Epithelial cells,454,0.06560636,False,0.05908944,False
AAACCTGAGACTGGGT-1-6,mysce,4431,1893,AAACCTGAGACTGGGT-1-6,E3,4.5464433,21.427018,nan,nan,nan,⋯,G1,E3,0.031799612,tS1,Epithelial cells,23,0.02200239,False,0.02101875,False
AAACCTGAGCACCGTC-1-6,mysce,6843,2399,AAACCTGAGCACCGTC-1-6,E3,13.8647687,14.434164,nan,nan,nan,⋯,G1,E3,0.027888148,tS1,Epithelial cells,356,0.05463441,False,0.05646173,False
AAACCTGAGCTAAACA_LUNG_T34,pubsce,11144,2479,AAACCTGAGCTAAACA_LUNG_T34,LUNG_T34,10.2992958,39.278169,Epithelial cells,Epithelial cells,tS1,⋯,G1,LUNG_T34,-0.006527102,tS1,Epithelial cells,255,0.03246239,False,0.03225806,False
AAACCTGAGGCTCTTA-1-6,mysce,6904,2650,AAACCTGAGGCTCTTA-1-6,E3,5.5524239,13.881060,nan,nan,nan,⋯,G1,E3,-0.032054523,tS1,Epithelial cells,37,0.03210176,False,0.02842377,False
AAACCTGCAAACTGCT-1-5,mysce,2903,1053,AAACCTGCAAACTGCT-1-5,E2,0.0660066,28.283828,nan,nan,nan,⋯,G1,E2,0.028680359,tS1,Epithelial cells,332,0.03149300,False,0.03225806,False


In [23]:
scesingleE <- as.SingleCellExperiment(sce)

In [24]:
scesingleE

class: SingleCellExperiment 
dim: 20532 12249 
metadata(0):
assays(2): counts logcounts
rownames(20532): A1BG A1BG-AS1 ... ZYX ZZEF1
rowData names(5): vst.mean vst.variance vst.variance.expected
  vst.variance.standardized vst.variable
colnames(12249): AAACCTGAGACGCTTT_LUNG_T18 AAACCTGAGACTGGGT-1-6 ...
  TTTGTCATCCAAAGTC_LUNG_T30 TTTGTCATCTTGCCGT_LUNG_T30
colData names(29): orig.ident nCount_RNA ... scrublet_predict_n30 ident
reducedDimNames(0):
altExpNames(0):

In [25]:
outDir <- file.path("output2_step2.5.3.filter_junk_scanpy_pipeline/data_Epithelial/malignant")
if(!dir.exists(outDir)) dir.create(outDir,recursive=TRUE)

In [27]:
save(sce, file=paste0(outDir, '/sceMalignant_cells.RData'))

In [28]:
save(sce, file=paste0(outDir, '/sceMalignant_cells_SingleCellExperiment.RData'))

In [30]:
mali_matrix=as.matrix(sce[['RNA']]@data)
mali_matrix[1:4,1:4]

,AAACCTGAGACGCTTT_LUNG_T18,AAACCTGAGACTGGGT-1-6,AAACCTGAGCACCGTC-1-6,AAACCTGAGCTAAACA_LUNG_T34
A1BG,0,0,0,0
A1BG-AS1,0,0,0,0
A1CF,0,0,0,0
A2M,0,0,0,0


In [32]:
mali_matrix=data.frame(gene_name= rownames(mali_matrix), mali_matrix)
mali_matrix[1:4,1:4]

,gene_name,AAACCTGAGACGCTTT_LUNG_T18,AAACCTGAGACTGGGT.1.6,AAACCTGAGCACCGTC.1.6
,<chr>,<dbl>,<dbl>,<dbl>
A1BG,A1BG,0,0,0
A1BG-AS1,A1BG-AS1,0,0,0
A1CF,A1CF,0,0,0
A2M,A2M,0,0,0


In [33]:
write.csv(mali_matrix, file=paste0(outDir, '/sceMalignant_cells_log_GEPmatrix.csv'), row.names=F)

In [36]:
suppressPackageStartupMessages({
    library("reticulate")
    library("ggplot2")
    #library("scater")
    library("Seurat")
})
sc <- import("scanpy")

In [37]:
####convert sce to h5ad
####
#sce=TotalTissue.combined
exprs <- GetAssayData(sce,assay.type = "RNA", slot = "counts")
meta <- sce[[]]
feature_meta <- GetAssay(sce)[[]]
#embedding <- Embeddings(sce, "umap")

aa=as.data.frame(as.matrix(exprs))
aa=t(aa)

colnames(meta)=gsub('[.]', "_", colnames(meta))
head(meta)

adata_seurat = sc$AnnData(X = aa, obs = meta, var = feature_meta)

Warning message:
"The following arguments are not used: assay.type"


,orig_ident,nCount_RNA,nFeature_RNA,Index,Sample,percent_mt,percent_rb,Cell_type,Cell_type_refined,Cell_subtype,⋯,Phase,old_ident,CC_Difference,Cell_subtype_unimodel,Cell_type_unimodel,UMAP_Clusters_harmony_overclustered,scrubletScore_default,scrublet_predict_default,scrubletScore_n30,scrublet_predict_n30
,<chr>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<fct>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<chr>,<dbl>,<chr>
AAACCTGAGACGCTTT_LUNG_T18,pubsce,5243,867,AAACCTGAGACGCTTT_LUNG_T18,LUNG_T18,0.1883594,1.130156,Epithelial cells,Epithelial cells,tS2,⋯,G1,LUNG_T18,-0.004011074,tS2,Epithelial cells,454,0.06560636,False,0.05908944,False
AAACCTGAGACTGGGT-1-6,mysce,4431,1893,AAACCTGAGACTGGGT-1-6,E3,4.5464433,21.427018,nan,nan,nan,⋯,G1,E3,0.031799612,tS1,Epithelial cells,23,0.02200239,False,0.02101875,False
AAACCTGAGCACCGTC-1-6,mysce,6843,2399,AAACCTGAGCACCGTC-1-6,E3,13.8647687,14.434164,nan,nan,nan,⋯,G1,E3,0.027888148,tS1,Epithelial cells,356,0.05463441,False,0.05646173,False
AAACCTGAGCTAAACA_LUNG_T34,pubsce,11144,2479,AAACCTGAGCTAAACA_LUNG_T34,LUNG_T34,10.2992958,39.278169,Epithelial cells,Epithelial cells,tS1,⋯,G1,LUNG_T34,-0.006527102,tS1,Epithelial cells,255,0.03246239,False,0.03225806,False
AAACCTGAGGCTCTTA-1-6,mysce,6904,2650,AAACCTGAGGCTCTTA-1-6,E3,5.5524239,13.881060,nan,nan,nan,⋯,G1,E3,-0.032054523,tS1,Epithelial cells,37,0.03210176,False,0.02842377,False
AAACCTGCAAACTGCT-1-5,mysce,2903,1053,AAACCTGCAAACTGCT-1-5,E2,0.0660066,28.283828,nan,nan,nan,⋯,G1,E2,0.028680359,tS1,Epithelial cells,332,0.03149300,False,0.03225806,False


In [38]:
adata_seurat

AnnData object with n_obs <U+00D7> n_vars = 12249 <U+00D7> 20532
    obs: 'orig_ident', 'nCount_RNA', 'nFeature_RNA', 'Index', 'Sample', 'percent_mt', 'percent_rb', 'Cell_type', 'Cell_type_refined', 'Cell_subtype', 'age', 'Sex', 'Smoking', 'Stages', 'percent_mt_merge', 'percent_rb_merge', 'S_Score', 'G2M_Score', 'Phase', 'old_ident', 'CC_Difference', 'Cell_subtype_unimodel', 'Cell_type_unimodel', 'UMAP_Clusters_harmony_overclustered', 'scrubletScore_default', 'scrublet_predict_default', 'scrubletScore_n30', 'scrublet_predict_n30'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'

In [39]:
adata_seurat$write(paste0(outDir, '/sceMalignant_cells.h5ad'))